In [1]:
import pygame 
pygame.init()

WIDTH, HEIGHT = 700,500
WIN = pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("Pong")
#bg = pygame.image.load("img1.jpg")
#WIN.blit(pygame.transform.scale(bg, (WIDTH, HEIGHT)), (0, 0))

FPS=60

PADDLE_WIDTH,PADDLE_HEIGHT = 20,100
RADIUS=7
WINNING_SCORE =10

SCORE_FONT = pygame.font.SysFont("comicsans",50)
FONT = pygame.font.SysFont("SFNT",70)

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WHITE = (255,255,255)
BLACK = (0,0,0)
RED = (255,0,0)
BLUE = (0,0,255)

In [3]:
def handle_paddle_movement(keys,left_paddle,right_paddle):
    if keys[pygame.K_w] and left_paddle.y-left_paddle.VEL >=0:
        left_paddle.move(up=True)
    if keys[pygame.K_s] and left_paddle.y+PADDLE_HEIGHT  + left_paddle.VEL <=HEIGHT:
        left_paddle.move(up=False)
    if keys[pygame.K_UP]  and right_paddle.y-right_paddle.VEL >=0:
        right_paddle.move(up=True)
    if keys[pygame.K_DOWN] and right_paddle.y +PADDLE_HEIGHT + right_paddle.VEL <=HEIGHT:
        right_paddle.move(up=False)

In [4]:
class Paddle:
    #COLOR=WHITE
    VEL=4
    
    def __init__ (self,x,y,width,height,color):
        self.x=self.original_x=x
        self.y=self.original_y=y
        self.width=width
        self.height=height
        self.color = color
        
    def draw(self,win):
        pygame.draw.rect(win,self.color,(self.x,self.y,self.width,self.height))
    def move(self,up=True):
        if up:
            self.y -=self.VEL
        else:
            self.y+=self.VEL
            
    def reset(self):
        self.x=self.original_x
        self.y=self.original_y

In [5]:
class Ball:
    MAX_VEL=5
    COLOR=WHITE
    
    def __init__(self,x,y,radius):
        self.x=self.original_x=x
        self.y=self.original_y = y
        self.radius=radius
        self.x_vel=self.MAX_VEL
        self.y_vel=0
        
    def draw(self,win):
        pygame.draw.circle(win,self.COLOR,(self.x,self.y),self.radius)
        
    def move(self):
        self.x+=self.x_vel
        self.y+=self.y_vel 
        
    def reset(self):
        self.x=self.original_x
        self.y =self.original_y
        self.y_vel=0
        self.x_vel*=-1
        

In [6]:
def handle_collision(ball,left_paddle,right_paddle):
    if ball.y+ball.radius >=HEIGHT:
        ball.y_vel *=-1
    if ball.y-ball.radius<=0:
        ball.y_vel *=-1
    
    if ball.x_vel <0:
        if ball.y >= left_paddle.y and ball.y<= left_paddle.y+left_paddle.height:
            if ball.x - ball.radius <= left_paddle.x+left_paddle.width:
                ball.x_vel *=-1
                
                middle_y = left_paddle.y + left_paddle.height//2
                difference = middle_y - ball.y
                reduction = (left_paddle.height/2) / ball.MAX_VEL
                y_vel = difference / reduction
                ball.y_vel = -1*y_vel
    else:
        if ball.y >= right_paddle.y and ball.y<= right_paddle.y+right_paddle.height:
            if ball.x +ball.radius >=right_paddle.x:
                ball.x_vel *=-1
                middle_y = right_paddle.y + right_paddle.height//2
                difference = middle_y - ball.y
                reduction = (right_paddle.height/2) / ball.MAX_VEL
                y_vel = difference / reduction
                ball.y_vel = -1*y_vel
        
                

In [7]:
def draw(win,paddles,ball,left_score,right_score):
    win.fill(BLACK)
    
    left_score_text = SCORE_FONT.render(f"{left_score}",1,RED)
    right_score_text = SCORE_FONT.render(f"{right_score}",1,BLUE)
    win.blit(left_score_text,(WIDTH//4- left_score_text.get_width()//2,20))
    win.blit(right_score_text,(WIDTH*3//4- right_score_text.get_width()//2,20))
    for paddle in paddles:
        paddle.draw(win)
        
    for i in range(10,HEIGHT,HEIGHT//20):
        if i%2==1:
            continue
        pygame.draw.rect(win,WHITE,(WIDTH//2-1,i,2,HEIGHT//20))
    
    ball.draw(win)
    pygame.display.update()
    

In [8]:
def main():
    run =True
    clock = pygame.time.Clock()
    left_paddle =Paddle(10,HEIGHT//2-PADDLE_HEIGHT//2,PADDLE_WIDTH,PADDLE_HEIGHT,RED)
    right_paddle =Paddle(WIDTH-10-PADDLE_WIDTH,HEIGHT//2-PADDLE_HEIGHT//2,
                         PADDLE_WIDTH,PADDLE_HEIGHT,BLUE)
    ball = Ball(WIDTH//2,HEIGHT//2,RADIUS)
    left_score =0
    right_score =0
    
    
    while run:
        clock.tick(FPS)
        draw(WIN,[left_paddle,right_paddle],ball,left_score,right_score)
        #WIN.blit(pygame.transform.scale(bg, (WIDTH, HEIGHT)), (0, 0))
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run = False
                break
        keys = pygame.key.get_pressed()
        handle_paddle_movement(keys,left_paddle,right_paddle)
        
        handle_collision(ball,left_paddle,right_paddle)
        ball.move()
        if ball.x <0:
            right_score+=1
            ball.reset()
            left_paddle.reset()
            right_paddle.reset()
        if ball.x >WIDTH:
            left_score +=1
            ball.reset()
            left_paddle.reset()
            right_paddle.reset()
            
        won =False
        if left_score >= WINNING_SCORE:
            win_text="Red Player Won"
            text = FONT.render(win_text,1,RED)
            won =True
            
        elif right_score >= WINNING_SCORE:
            win_text="Blue Player Won"
            text = FONT.render(win_text,1,BLUE)
            won=True
            
        
        if won:
            WIN.blit(text,(WIDTH//2-text.get_width()//2,HEIGHT//2-text.get_height()//2))
            pygame.display.update()
            pygame.time.delay(5000)
            ball.reset()
            left_paddle.reset()
            right_paddle.reset()
            left_score=0
            right_score=0
        
        
    pygame.quit()

In [9]:
if __name__=='__main__':
    main()